## Share Tensor

In [14]:
import syft as sy
import torch

from sympc.session import Session, SessionManager
from sympc.tensor import MPCTensor

sy.load("sympc")
sy.logger.add(sink="./example.log")

### Create the Clients

In [15]:
alice = sy.VirtualMachine(name="alice")
bob = sy.VirtualMachine(name="bob")

alice_client = alice.get_client()
bob_client = bob.get_client()

### Create Session

In [17]:
session = Session(parties=[alice_client, bob_client])
print(session)

### Send the session to all the parties

In [18]:
SessionManager.setup_mpc(session)

In [20]:
x = MPCTensor(secret=torch.tensor([1,2,3,4]), session=session)

In [21]:
x

[MPCTensor]
Shape: torch.Size([4])
Requires Grad: False
	| <VirtualMachineClient: alice Client> -> ShareTensorPointer
	| <VirtualMachineClient: bob Client> -> ShareTensorPointer

In [22]:
for val in x.share_ptrs:
    print(val.get()) # get destroys the pointer 
    # .reconstruct() also destroys the pointer

[ShareTensor]
	| [FixedPointEncoder]: precision: 16, base: 2
	| Data: tensor([1008230258186094461, 2895831840715505846,  881793063096377731,
        2143416801702356975])
[ShareTensor]
	| [FixedPointEncoder]: precision: 16, base: 2
	| Data: tensor([-1008230258186028925, -2895831840715374774,  -881793063096181123,
        -2143416801702094831])


### Secret is shared by orhestrator

In [27]:
x_secret = torch.tensor([[1,2],[3,4]])
x = MPCTensor(secret=x_secret, session=session)

y = torch.tensor([[5,6],[7,8]])

In [28]:
print("[Priv + Pub] X + Y =\n", (x + y).reconstruct())
print("[Pub + Pub] X + Y =\n", x_secret + y)

[Priv + Pub] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])
[Pub + Pub] X + Y =
 tensor([[ 6,  8],
        [10, 12]])


In [29]:
print("[Priv - Pub] X - Y =\n", (x - y).reconstruct())
print("[Pub - Pub] X - Y =\n", x_secret - y)

[Priv - Pub] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])
[Pub - Pub] X - Y =
 tensor([[-4, -4],
        [-4, -4]])


In [30]:
print("[Priv * Pub] X * Y =\n", (x * y).reconstruct())
print("[Pub * Pub] X * Y =\n", x_secret * y)

[Priv * Pub] X * Y =
 tensor([[ 5., 12.],
        [21., 32.]])
[Pub * Pub] X * Y =
 tensor([[ 5, 12],
        [21, 32]])


In [31]:
print("[Priv @ Pub] X * Y =\n", (x @ y).reconstruct())
print("[Pub @ Pub] X * Y =\n", x_secret @ y)

[Priv @ Pub] X * Y =
 tensor([[19., 22.],
        [43., 50.]])
[Pub @ Pub] X * Y =
 tensor([[19, 22],
        [43, 50]])


### Private Operations

In [33]:
x_secret = torch.tensor([[1,2],[3,4]])
x = MPCTensor(secret=x_secret, session=session)

y_secret = torch.tensor([[5,6],[7,8]])
y = MPCTensor(secret=y_secret, session=session)


In [36]:
print("[Priv + Priv] X + Y =\n", (x + y).reconstruct())
print("[Pub + Pub] X + Y =\n", x_secret + y_secret)

[Priv + Priv] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])
[Pub + Pub] X + Y =
 tensor([[ 6,  8],
        [10, 12]])


In [37]:
print("[Priv - Priv] X - Y =\n", (x - y).reconstruct())
print("[Pub - Pub] X - Y =\n", x_secret - y_secret)

[Priv - Priv] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])
[Pub - Pub] X - Y =
 tensor([[-4, -4],
        [-4, -4]])


In [41]:
#print("[Priv * Priv] X * Y =\n", (x * y).reconstruct()) 
# causes next error:
# Request to access data length rejected.

print("[Pub * Pub] X * Y =\n", x_secret * y_secret)

[Pub * Pub] X * Y =
 tensor([[ 5, 12],
        [21, 32]])


In [43]:
#print("[Priv @ Pub] X * Y =\n", (x @ y).reconstruct())
# causes next error:
# Request to access data length rejected.

print("[Pub @ Pub] X * Y =\n", x_secret @ y_secret)

[Pub @ Pub] X * Y =
 tensor([[19, 22],
        [43, 50]])
